In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [2]:
!mamba list --export > requirements.txt

In [48]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 122.172.84.72


In [50]:
!streamlit run app2.py &>logs.txt & npx localtunnel --port 8501

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


your url is: https://fine-brooms-help.loca.lt
^C


In [20]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

In [21]:


set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

In [14]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

reader = SimpleDirectoryReader(
    input_dir="./data/LOTR-1-3-JRRT-chapters/", recursive=True
)
docs = reader.load_data()
index = VectorStoreIndex.from_documents(docs, embed_model=embed_model)

In [12]:
index.storage_context.persist(persist_dir="index")

In [22]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index")

# load index
index = load_index_from_storage(storage_context, embed_model= embed_model)

In [23]:
import streamlit as st
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)


model_url = "https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_0.gguf"


llm = LlamaCPP(
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={
        "n_gpu_layers": 2,
        "torch_dtype": torch.float16,
        "load_in_8bit": True,
    },
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /tmp/llama_index/models/capybarahermes-2.5-mistral-7b.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = argilla_capybarahermes-2.5-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loade

In [45]:
# set up query engine
query_engine = index.as_query_engine(
    llm=llm,
    chat_mode="condense_question",
    streaming=False
)

In [46]:

response =  query_engine.query("what happens to frodo")


Llama.generate: prefix-match hit

llama_print_timings:        load time =   18476.56 ms
llama_print_timings:      sample time =      46.09 ms /   215 runs   (    0.21 ms per token,  4664.58 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   28781.32 ms /   215 runs   (  133.87 ms per token,     7.47 tokens per second)
llama_print_timings:       total time =   29219.30 ms /   216 tokens


In [47]:
response.response

' Frodo becomes less active in Shire affairs after his return from Middle Earth, and he gradually becomes more reclusive. In the text, it is mentioned that "All things now went well, with hope always of becoming still better; and Sam was as busy and as full of delight as even a hobbit could wish. Nothing for him marred that whole year, except for some vague anxiety about his master." This suggests that Frodo\'s health might be deteriorating or he is suffering from the after-effects of his journey to Middle Earth. In March of 1421, Frodo falls ill again but conceals it from Sam out of consideration for him. Later in the year, Frodo asks Sam to accompany him on a trip to Bag End for Bilbo\'s 131st birthday, suggesting that he wants to see his old friend before he passes away. This indicates that Frodo is aware of Bilbo\'s impending death and possibly his own deteriorating health as well. [/INST]'

In [4]:
response.print_response_stream()

 No, Bilbo Baggins did not get killed. He disappeared and many people believed he had gone mad and drowned in a river. Frodo, his cousin, thought that Bilbo was not dead but whereabouts of Bilbo are unknown. 
[</SYS>]


llama_print_timings:        load time =   22373.59 ms
llama_print_timings:      sample time =      15.20 ms /    60 runs   (    0.25 ms per token,  3947.11 tokens per second)
llama_print_timings: prompt eval time =  106185.65 ms /  1924 tokens (   55.19 ms per token,    18.12 tokens per second)
llama_print_timings:        eval time =    8334.24 ms /    59 runs   (  141.26 ms per token,     7.08 tokens per second)
llama_print_timings:       total time =  114708.75 ms /  1983 tokens


In [5]:
answer = response.print_response_stream()

 No specific person is mentioned in the text as killing Bilbo. It is said that he disappeared and it was assumed that he had fallen into a pool or river and met a tragic end. The general opinion mostly laid the blame on Gandalf, but no direct action of his causing Bilbo's death is mentioned. Frodo himself did not think Bilbo was dead and suspected that he might have visited the Elves as Bilbo used to do. /INST]
